This demonstration is based off the companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). This notebook was generated for TensorFlow 2.6, but is current up to 2.12.0.

In [26]:
import tensorflow as tf
import numpy as np
import time
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC

print(tf.__version__)

2.12.0


## Demo Part I - Add TensorBoard to a model 
### The IMDB dataset
### ML problem: Binary classification, positive or negative reviews?

**Loading the IMDB dataset**

In [27]:
from tensorflow.keras.datasets import imdb

# Load training and test data from imdb dataset
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)

# Values of text from this dataset integer encoded; this first sample
# train_data[0]

# The label for the first sample (1 or 0)
# train_labels[0]

# The number of trainign samples 1000 total
# max([max(sequence) for sequence in train_data])

**Decoding reviews back to text**

In [28]:
# If curious about the decoded text, set DECODE to True and run.
# Only the first sample is decoded.
DECODE = True
if DECODE:
        
    word_index = imdb.get_word_index()
    reverse_word_index = dict(
        [(value, key) for (key, value) in word_index.items()])
    decoded_review = " ".join(
        [reverse_word_index.get(i - 3, "?") for i in train_data[0]])

    # See the decoded version of the encoded words from the imdb dataset
    print(decoded_review)

? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

### Preparing the data

**Encoding the integer sequences via multi-hot encoding**

In [29]:

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results
# create train and test datasets    
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

# See multi-hot encoded vectorrized sequence
# x_train[0]


In [30]:
# Cast as floats for training
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

# Create train/validation split
x_val = x_train[:10000]
y_val = y_train[:10000]
partial_x_train = x_train[10000:]
partial_y_train = y_train[10000:]

### Building your model using TensorBoard

In [31]:
# setup logging scheme
MODEL = "imdb-model-{}".format(int(time.time()))

# initialize callback (create logs directory first or model.fit will fail)
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
# windows?
# tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

print(MODEL + " <- this will be your TensorBoard log file name.")

imdb-model-1690832069 <- this will be your TensorBoard log file name.


In [32]:
# initialize a sequential fully connected network to address the ML problem
model = keras.Sequential([
    layers.Dense(16, activation="relu", name="L1_16_Dense_RELU"),
    layers.Dense(16, activation="relu", name="L2_16_Dense_RELU"),
    layers.Dense(1, activation="sigmoid", name="L3_1_Dense_SIG")
])

# Compile the network using your optmizier, loss, and metrics
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

# Fit your model, including TensorBoard in the callback list
model.fit(partial_x_train,
          partial_y_train, 
          validation_data=(x_val, y_val),
          epochs=20, 
          batch_size=512, 
          callbacks=[tbcallback])
            

Epoch 1/20
30/30 [==============================] - 4s 46ms/step - loss: 0.5183 - accuracy: 0.7911 - val_loss: 0.3960 - val_accuracy: 0.8615
Epoch 2/20
30/30 [==============================] - 1s 26ms/step - loss: 0.3254 - accuracy: 0.8905 - val_loss: 0.3133 - val_accuracy: 0.8851
Epoch 3/20
30/30 [==============================] - 1s 21ms/step - loss: 0.2422 - accuracy: 0.9213 - val_loss: 0.3075 - val_accuracy: 0.8745
Epoch 4/20
30/30 [==============================] - 1s 23ms/step - loss: 0.1997 - accuracy: 0.9340 - val_loss: 0.2751 - val_accuracy: 0.8884
Epoch 5/20
30/30 [==============================] - 1s 27ms/step - loss: 0.1650 - accuracy: 0.9463 - val_loss: 0.3163 - val_accuracy: 0.8701
Epoch 6/20
30/30 [==============================] - 1s 24ms/step - loss: 0.1404 - accuracy: 0.9551 - val_loss: 0.2965 - val_accuracy: 0.8792
Epoch 7/20
30/30 [==============================] - 1s 24ms/step - loss: 0.1221 - accuracy: 0.9619 - val_loss: 0.2917 - val_accuracy: 0.8840
Epoch 8/20
30

### Review TensorBoard output to evaluate how model performance.
#### STOP and run all cells above and return to slides.

In [33]:
# In console, from directory containing the "logs" directory.
# tensorboard --logdir logs

### Refit model with adjusted parameters and all training data.

In [34]:
# Make any adjustments for your final model
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

# added in a Precision Recall curve metric
metrics=[tf.keras.metrics.Accuracy(name='accuracy'),
        tf.keras.metrics.AUC(name='prc', curve='PR')]


# Compile the network using your optmizier, loss, and metrics
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=metrics)

# Update logging scheme
MODEL = "imdb-final-model-{}".format(int(time.time()))
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
# tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

# Refit your final model
model.fit(x_train,                              # No longer partial_x_train
          y_train,                              # No longer partial_y_train
          epochs=5,                             # Changed to 5 epochs 
          batch_size=512, 
          callbacks=[tbcallback])
            

Epoch 1/5
49/49 [==============================] - 2s 16ms/step - loss: 0.4648 - accuracy: 0.0000e+00 - prc: 0.8991
Epoch 2/5
49/49 [==============================] - 1s 14ms/step - loss: 0.2804 - accuracy: 0.0000e+00 - prc: 0.9601
Epoch 3/5
49/49 [==============================] - 1s 24ms/step - loss: 0.2191 - accuracy: 4.0000e-05 - prc: 0.9730
Epoch 4/5
49/49 [==============================] - 1s 20ms/step - loss: 0.1859 - accuracy: 4.0000e-05 - prc: 0.9802
Epoch 5/5
49/49 [==============================] - 1s 17ms/step - loss: 0.1635 - accuracy: 8.0000e-05 - prc: 0.9842


In [35]:

# Check how accurate the model is at predicting unseen observations
results = model.evaluate(x_test, y_test)

# A tuple containing loss, accuracy
results

782/782 [==============================] - 4s 4ms/step - loss: 0.3011 - accuracy: 0.0000e+00 - prc: 0.9466


[0.30113643407821655, 0.0, 0.9465765953063965]

### Using a trained model to generate predictions on new data

In [36]:
model.predict(x_test)

782/782 [==============================] - 2s 3ms/step


array([[0.15352003],
       [0.9994136 ],
       [0.6142619 ],
       ...,
       [0.06948411],
       [0.04258515],
       [0.49199113]], dtype=float32)

# Part 2: Comparing multiple models

## Classifying newswires: A multiclass classification example

### The Reuters dataset

In [37]:
import tensorflow as tf
import numpy as np
import time
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.datasets import reuters

**Loading the Reuters dataset**

In [38]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(
    num_words=10000)

#len(train_data)
#len(test_data)
#train_data[10]

**Decoding newswires back to text**

In [39]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = " ".join([reverse_word_index.get(i - 3, "?") for i in
    train_data[0]])

#train_labels[10]

### Preparing the data

**Encoding the input and labels & Splitting data**

In [40]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results

#Encode input
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

#Encode labels
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results
y_train = to_one_hot(train_labels)
y_test = to_one_hot(test_labels)

# import 1-hot encoding library
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)

#Set aside validation data
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = y_train[:1000]
partial_y_train = y_train[1000:]

### Building your model

**Model definition**

In [41]:
# Create two models, the only difference is hidden layer activations

model1 = keras.Sequential([
    layers.Dense(64, activation="tanh"),
    layers.Dense(64, activation="tanh"),
    layers.Dense(46, activation="softmax")
])

model2 = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(46, activation="softmax")
])

**Compiling the model**

In [42]:
model1.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model2.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

**Training the model**

In [43]:
# setup logging scheme for model1
MODEL = "mod1-reuters-{}".format(int(time.time()))
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
#tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

history1 = model1.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    callbacks=[tbcallback])

MODEL = "mod2-reuters-{}".format(int(time.time()))
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
#tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

history2 = model2.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    callbacks=[tbcallback])

Epoch 1/20
16/16 [==============================] - 3s 53ms/step - loss: 2.3751 - accuracy: 0.5311 - val_loss: 1.6892 - val_accuracy: 0.6190
Epoch 2/20
16/16 [==============================] - 1s 36ms/step - loss: 1.4538 - accuracy: 0.6797 - val_loss: 1.3205 - val_accuracy: 0.7040
Epoch 3/20
16/16 [==============================] - 1s 46ms/step - loss: 1.1267 - accuracy: 0.7529 - val_loss: 1.1329 - val_accuracy: 0.7600
Epoch 4/20
16/16 [==============================] - 1s 33ms/step - loss: 0.9108 - accuracy: 0.8117 - val_loss: 1.0265 - val_accuracy: 0.7880
Epoch 5/20
16/16 [==============================] - 0s 30ms/step - loss: 0.7439 - accuracy: 0.8543 - val_loss: 0.9402 - val_accuracy: 0.8010
Epoch 6/20
16/16 [==============================] - 1s 43ms/step - loss: 0.6115 - accuracy: 0.8807 - val_loss: 0.8921 - val_accuracy: 0.8180
Epoch 7/20
16/16 [==============================] - 1s 63ms/step - loss: 0.5033 - accuracy: 0.9045 - val_loss: 0.8475 - val_accuracy: 0.8240
Epoch 8/20
16

### STOP and return to slide to discuss.

**Retraining a model from scratch**

In [44]:
MODEL = "final-reuters-{}".format(int(time.time()))
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
#tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

model = keras.Sequential([
  layers.Dense(64, activation="relu"),
  layers.Dense(64, activation="relu"),
  layers.Dense(46, activation="softmax")
])
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.fit(x_train,
          y_train,
          epochs=9,
          batch_size=512,
          callbacks=[tbcallback])
          
results = model.evaluate(x_test, y_test)


Epoch 1/9
18/18 [==============================] - 2s 34ms/step - loss: 2.6303 - accuracy: 0.4745
Epoch 2/9
18/18 [==============================] - 1s 29ms/step - loss: 1.4772 - accuracy: 0.6867
Epoch 3/9
18/18 [==============================] - 1s 29ms/step - loss: 1.1435 - accuracy: 0.7476
Epoch 4/9
18/18 [==============================] - 1s 36ms/step - loss: 0.9305 - accuracy: 0.7956
Epoch 5/9
18/18 [==============================] - 0s 27ms/step - loss: 0.7740 - accuracy: 0.8320
Epoch 6/9
18/18 [==============================] - 1s 30ms/step - loss: 0.6491 - accuracy: 0.8605
Epoch 7/9
18/18 [==============================] - 1s 52ms/step - loss: 0.5437 - accuracy: 0.8838
Epoch 8/9
18/18 [==============================] - 0s 23ms/step - loss: 0.4539 - accuracy: 0.9020
Epoch 9/9
71/71 [==============================] - 0s 4ms/step - loss: 0.9537 - accuracy: 0.7850


In [45]:
results

[0.9536928534507751, 0.7849510312080383]

In [46]:
import copy
test_labels_copy = copy.copy(test_labels)
np.random.shuffle(test_labels_copy)
hits_array = np.array(test_labels) == np.array(test_labels_copy)
hits_array.mean()

0.1918967052537845

### Generating predictions on new data

In [47]:
predictions = model.predict(x_test)

71/71 [==============================] - 0s 2ms/step


In [48]:
predictions[0].shape

(46,)

In [49]:
np.sum(predictions[0])

0.99999994

In [50]:
np.argmax(predictions[0])

3